In [8]:
#pip install aiohttp asyncssh

In [9]:
import asyncio
import asyncssh
import aiohttp
from http.client import HTTPConnection
from urllib.parse import urlparse


async def site_is_online_async(url, timeout):
    """Return True if the target URL is online.

    Raise an exception otherwise.
    """
    error = Exception("unknown scheme")
    
    parser = urlparse(url)
    host = parser.netloc or parser.path.split("/")[0]
    
    
    if parser.scheme in ("http", "https"):
        async with aiohttp.ClientSession() as session:
            try:
                await session.head(url, timeout=timeout,ssl=False)
                return True
            except asyncio.TimeoutError:
                error = Exception("timed out")
            except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("ssh"):
        if  parser.port == None : 
            port=22 
        else: 
            port=parser.port
            host=host.split(":")[0]

        #print(host,port)
        
        try:
            async with await asyncio.wait_for(asyncssh.connect(host=host,port=port,username="test", password="test", known_hosts=None), timeout=timeout) as conn:
                print((await conn.run('tty', term_type='ansi')).stdout, end='')
                return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("postgre"):                
        if  parser.port == None : 
            port=5432
        else: 
            port=parser.port
            host=host.split(":")[0]
        
        coro = asyncio.open_connection(host, port)
        # execute the coroutine with a timeout
        try:
            # open the connection and wait for a moment
            _,writer = await asyncio.wait_for(coro, timeout)
            # close connection once opened
            writer.close()
            # indicate the connection can be opened
            return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("kafka"):                
        if  parser.port == None : 
            port=9092
        else: 
            port=parser.port
            host=host.split(":")[0]
        
        coro = asyncio.open_connection(host, port)
        # execute the coroutine with a timeout
        try:
            # open the connection and wait for a moment
            _,writer = await asyncio.wait_for(coro, timeout)
            # close connection once opened
            writer.close()
            # indicate the connection can be opened
            return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
                
    raise error
    
def display_check_result(result, url, error=""):
    """Display the result of a connectivity check."""
    print(f'The status of "{url}" is:', end=" ")
    if result:
        print('"Online!" 👍')
    else:
        print(f'"Offline?" 👎 \n  Error: "{error}"')


async def _asynchronous_check(urls,timeout):
    async def _check(url,timeout):
        error = ""
        try:
            result = await site_is_online_async(url,timeout)
        except Exception as e:
            result = False
            error = str(e)
        display_check_result(result, url, error)

    await asyncio.gather(*(_check(url,timeout) for url in urls))

def check(urls,timeout=5):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        loop = None

    if loop and loop.is_running():
        print('Async event loop already running. Adding coroutine to the event loop.')
        tsk = loop.create_task(_asynchronous_check(urls,timeout))
        # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
        # Optionally, a callback function can be executed when the coroutine completes
        tsk.add_done_callback(
            lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
    else:
        print('Starting new event loop')
        result = asyncio.run(_asynchronous_check(urls,timeout))


In [10]:
import time
time.sleep(1)
check(timeout=3,urls=[])

Async event loop already running. Adding coroutine to the event loop.


In [11]:
check(timeout=5, urls=["kafka://scanme.nmap.org",
                       "kafka://scanme.nmap.org:9092",
                       "kafka://scanme.nmap.org:9093",
                       "postgre://scanme.nmap.org",
                       "http://scanme.nmap.org",
                       "http://scanme.nmap.org/",
                       "http://scanme.nmap.org:80",
                       "http://scanme.nmap.org:8080",
                       "https://scanme.nmap.org",
                       "https://scanme.nmap.org:443",
                       "ssh://scanme.nmap.org",
                       "ssh://scanme.nmap.org",
                       "ssh://scanme.nmap.org:22",
                       "ssh://scanme.nmap.org:2222"
                      ]
     )

Task done with result=None  << return val of main()
Async event loop already running. Adding coroutine to the event loop.
The status of "https://scanme.nmap.org" is: "Offline?" 👎 
  Error: "Cannot connect to host scanme.nmap.org:443 ssl:False [Connect call failed ('45.33.32.156', 443)]"
The status of "kafka://scanme.nmap.org:9093" is: "Offline?" 👎 
  Error: "Multiple exceptions: [Errno 111] Connect call failed ('45.33.32.156', 9093), [Errno 99] Cannot assign requested address"
The status of "kafka://scanme.nmap.org" is: "Offline?" 👎 
  Error: "Multiple exceptions: [Errno 111] Connect call failed ('45.33.32.156', 9092), [Errno 99] Cannot assign requested address"
The status of "postgre://scanme.nmap.org" is: "Offline?" 👎 
  Error: "Multiple exceptions: [Errno 111] Connect call failed ('45.33.32.156', 5432), [Errno 99] Cannot assign requested address"
The status of "ssh://scanme.nmap.org:2222" is: "Offline?" 👎 
  Error: "Multiple exceptions: [Errno 111] Connect call failed ('45.33.32.156